# Deep Learning - Tutorial Facial Keypoints Detection
### Magister en Data Science

#### Instituto Data Science, Universidad del Desarrollo

By Hernan Rivera 

* Tutorial esta basado en el tutorial de Daniel Nouri fundadod de Natural Vision. 
* Original: http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial

Tutorial Contents

* Requisitos previos
* Los datos
* Primer modelo: una sola capa oculta.
* Probándolo
* Segundo modelo: convolutions.
* Aumento de datos
* Cambio de la tasa de aprendizaje y el impulso con el tiempo
* Abandonar
* Especialistas en formación.
* Pre-entrenamiento supervisado.
* Conclusión

### 1. Requisitos previos

* Se debe instalar las librerias Theano y lasagne

In [2]:
#!pip install -r https://raw.githubusercontent.com/dnouri/kfkd-tutorial/master/requirements.txt
# En mi caso previamente instalada, por lo tanto, se comenta el comando.

### 2. Los datos

* Cremoa funcion para chequear los datos dispoibles para el desafio

In [4]:
import os
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

* Los datos se descargan desde el sitio web de kagle (https://www.kaggle.com/c/facial-keypoints-detection/data)

In [7]:
FTRAIN = './data/training.csv'
FTEST = './data/test.csv'

In [8]:
def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

In [9]:
X, y = load()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

* Podemos observar las caractetisticas principales del dataset, en donde destaca lo siguiente:
 > 
 * La data de entrenamineto consta de 7049 imagenes de 96 x 96 pixeles en escala de grises.
 * Existen 30 caracteristicas que corresponden a los pares (puntos) X, Y de los 15 puntos claves de la cara.
 * Para algunos puntos claves solo tenemso aprox 2000 etiquetas, mientras que para otros cerca de 7000 etiquetas.
 * **y.shape == (2140, 30)** nos dice que solo hay 2140 imagenes en el dataset con todas las features
 * Las imagenes estan en escala [0,1] en vez de [0, 255]
 * Los valores de X, Y estan entre [-1,1] y las imagenes de [0,95]

### 3. Primer modelo: una sola capa oculta.

* Creamos una red Lasagne con una solo una capa:
 > 
 * Primero inicializamos la red con 3 capas (input, hidden y output). En este punto se especifica el nombre y su orden.
 * Luego se especifica las caracteristicas de cada capa
 * 

In [14]:
net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1, # Print 1, Not print 0
    )

In [15]:
net1.fit(X, y)

# Neural Network with 924730 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input     9216
  1  hidden     100
  2  output      30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.10887     0.09718    1.12033  0.32s
      2     0.04194     0.01798    2.33336  0.29s
      3     0.01648     0.01645    1.00192  0.30s
      4     0.01476     0.01476    1.00007  0.30s
      5     0.01354     0.01382    0.97974  0.30s
      6     0.01276     0.01314    0.97139  0.29s
      7     0.01214     0.01257    0.96633  0.29s
      8     0.01163     0.01209    0.96236  0.28s
      9     0.01121     0.01168    0.95928  0.29s
     10     0.01084     0.01134    0.95530  0.29s
     11     0.01052     0.01105    0.95251  0.31s
     12     0.01025     0.01079    0.95001  0.32s
     13     0.01000     0.01055    0.94759  0.34s
     14     0.00977     0.01034    0.94553  0.26s
     15     0.00957     0.01

    118     0.00430     0.00517    0.83257  0.28s
    119     0.00428     0.00515    0.83171  0.29s
    120     0.00426     0.00513    0.83089  0.27s
    121     0.00425     0.00512    0.83005  0.26s
    122     0.00423     0.00510    0.82916  0.26s
    123     0.00421     0.00508    0.82833  0.27s
    124     0.00419     0.00507    0.82744  0.27s
    125     0.00418     0.00505    0.82667  0.26s
    126     0.00416     0.00504    0.82573  0.26s
    127     0.00414     0.00502    0.82495  0.26s
    128     0.00412     0.00500    0.82412  0.25s
    129     0.00411     0.00499    0.82318  0.28s
    130     0.00409     0.00497    0.82237  0.28s
    131     0.00407     0.00496    0.82152  0.28s
    132     0.00406     0.00494    0.82074  0.28s
    133     0.00404     0.00493    0.81987  0.28s
    134     0.00402     0.00491    0.81909  0.27s
    135     0.00401     0.00490    0.81844  0.26s
    136     0.00399     0.00488    0.81757  0.27s
    137     0.00398     0.00487    0.81670  0.26s


    239     0.00287     0.00389    0.73919  0.27s
    240     0.00286     0.00388    0.73832  0.25s
    241     0.00286     0.00387    0.73760  0.26s
    242     0.00285     0.00387    0.73701  0.26s
    243     0.00284     0.00386    0.73625  0.26s
    244     0.00284     0.00385    0.73561  0.25s
    245     0.00283     0.00385    0.73485  0.26s
    246     0.00282     0.00384    0.73420  0.25s
    247     0.00281     0.00384    0.73345  0.26s
    248     0.00281     0.00383    0.73284  0.26s
    249     0.00280     0.00382    0.73206  0.26s
    250     0.00279     0.00382    0.73131  0.26s
    251     0.00279     0.00381    0.73071  0.26s
    252     0.00278     0.00381    0.72989  0.26s
    253     0.00277     0.00380    0.72934  0.26s
    254     0.00277     0.00379    0.72874  0.26s
    255     0.00276     0.00379    0.72790  0.26s
    256     0.00275     0.00378    0.72741  0.26s
    257     0.00275     0.00378    0.72657  0.26s
    258     0.00274     0.00377    0.72602  0.26s


    370     0.00222     0.00335    0.66268  0.35s
    371     0.00221     0.00334    0.66085  0.30s
    372     0.00220     0.00334    0.65948  0.31s
    373     0.00220     0.00333    0.65920  0.42s
    374     0.00220     0.00333    0.65933  0.33s
    375     0.00219     0.00333    0.65854  0.30s
    376     0.00219     0.00333    0.65755  0.30s
    377     0.00218     0.00333    0.65670  0.29s
    378     0.00218     0.00333    0.65619  0.32s
    379     0.00218     0.00332    0.65590  0.30s
    380     0.00217     0.00332    0.65537  0.30s
    381     0.00217     0.00331    0.65485  0.30s
    382     0.00217     0.00331    0.65422  0.34s
    383     0.00216     0.00331    0.65385  0.34s
    384     0.00216     0.00331    0.65333  0.30s
    385     0.00216     0.00330    0.65285  0.31s
    386     0.00215     0.00330    0.65222  0.30s
    387     0.00215     0.00330    0.65164  0.30s
    388     0.00215     0.00330    0.65113  0.29s
    389     0.00214     0.00330    0.65079  0.30s


NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x1114c1ef0>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x1114c1eb8>,
     check_input=True, custom_scores=None, hidden_num_units=100,
     input_shape=(None, 9216),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=400, more_params={},
     objective=<function objective at 0x1116a31e0>,
     objective_loss_function=<function squared_error at 0x1113da1e0>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog object at 0x1c29f02908>],
     on_training_finished=[],
     on_training_started=[<nolearn.lasagne.handlers.PrintLayerInfo object at 0x1c29f02940>],
     output_nonlinearity=None, output_num_units=30, regression=True,
     scores_train=[], scores_valid=[],
     train_s

* Comparación de metodos de optimización(by Alec Radford). La estrella denota el minimo global en el plano de error. Nosotros usaremos Nesterov's Accelerated Gradient Descent (NAG) en este tutorial.

<img src="img/methods.gif" />